In [42]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import json
from bertopic import BERTopic
from hdbscan import HDBSCAN
import re

c:\Users\真希\Downloads\nogi_tweet\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import os
os.environ["AWS_BEARER_TOKEN_BEDROCK"] = "ABSKQmVkcm9ja0FQSUtleS03MjFrLWF0LTk5MjM4MjYxMjQ3NDpJMjNuVjk4VVBPbXY0MVZWaUJXSnViVlQ2WFJ2ZlVJT2FoRmZ4S2w5TGwxVXZzNkpZNEVRS05tMVdsaz0="

In [43]:
json_open = open("nogizaka_tweets.json","r",encoding="utf-8")
json_open = json.load(json_open)

id = []
text = []
viewCount = []
likeCount = []
url = []

for i in range(len(json_open)):
    id.append(json_open[i]["id"])
    text.append(json_open[i]["text"])
    viewCount.append(json_open[i]["viewCount"])
    likeCount.append(json_open[i]["likeCount"])
    url.append(json_open[i]["url"])

df = pd.DataFrame({
    "id":id,
    "text":text,
    "viewCount":viewCount,
    "likeCount":likeCount,
    "url":url})

df["text"] = df["text"].str.replace("\n","")
pattern = r'https?://[^\s]+'
df["text"] = df["text"].apply(lambda x:re.sub(pattern,"",x))
df

,id,text,viewCount,likeCount,url
0,1970330163209789748,＼絶賛発売中‼️／#GoodsPress 10月号連載一周年を記念してリバーシブル表紙に登場...,50307,2305,https://x.com/GoodsPress/status/19703301632097...
1,1967592475046097140,色んな想いの人がいると思うけど、自分はアイドルが好きなんじゃなくて乃木坂46が大好き,243350,4181,https://x.com/ikuma_nogi/status/19675924750460...
2,1969265264924532742,乃木坂46×So-netコラボキャンペーンサイトでは、CM撮影時のオフショットやメンバーから...,205970,6042,https://x.com/Sonet_fukuonsei/status/196926526...
3,1968646188762316907,次回の #乃木坂あそぶだけ は...NOGIZAKA ROOM SHARE 🏠** ┈┈┈┈...,568518,6838,https://x.com/nogidoga46/status/19686461887623...
4,1970146155725312331,【今日は何の日】9月23日の乃木坂46,895,22,https://x.com/nogizaka46or/status/197014615572...
...,...,...,...,...,...
195,1969421340508897381,【今日は何の日】9月21日の乃木坂46,688,24,https://x.com/nogidic3/status/1969421340508897381
196,1968347566548148699,【今日は何の日】9月18日の乃木坂46 知ってて当然!!! ボーダークイズ 後半戦』#齋藤飛鳥,6963,227,https://x.com/asukabiiki/status/19683475665481...
197,1966857129631080478,"#過剰接続 noteメンバーシップ限定音声｢Tyler, The Creator来日公演と乃...",32058,11,https://x.com/kimu_ra10/status/196685712963108...
198,1966807568992661877,『6th Anniversary 乃木坂46時間TV』を忘れない画像,3685,199,https://x.com/nogizaka46or/status/196680756899...


### まずは基本のクエリ
参考1：#基本のコード呼び出し：https://docs.aws.amazon.com/ja_jp/bedrock/latest/userguide/inference-invoke.html
<br>
参考2:#anthropicモデルの使い方：https://docs.aws.amazon.com/ja_jp/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html

In [ ]:
import boto3
import json
from botocore.exceptions import ClientError

client = boto3.client(
    "bedrock-runtime",
    region_name="us-east-1"
)

prompt = "次の文章を内容に基づいてサマリーを出してください。①メンバーに関する内容乃木坂46×So-netコラボキャンペーンサイトでは、CM撮影時のオフショットやメンバーから②『6th Anniversary 乃木坂46時間TV』を忘れない画像"

body = json.dumps({
    "messages":[
        {"role":"user","content":prompt}
    ],
    "max_tokens":512,
    "temperature":0.5,
    "anthropic_version":"bedrock-2023-05-31"
    
})

model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"

response = client.invoke_model(
    modelId=model_id,
    body=body
)

response_body = json.loads(response.get("body").read())
answer = response_body["content"][0]["text"]
print(answer)


このテキストには、乃木坂46に関する2つの主要な情報が含まれています：

1. 乃木坂46とSo-netのコラボキャンペーンサイトについて：
   - CMの撮影時のオフショット画像が公開されている
   - メンバーからの何らかの情報や内容が含まれている（具体的な内容は不明）

2. 『6th Anniversary 乃木坂46時間TV』に関連する画像の存在

これらの情報は、乃木坂46のファン向けのコンテンツやイベントに関するものであり、グループの活動や記念イベントの一部を紹介しているようです。


### Langchainを利用<br>
参考：https://qiita.com/naito_kc/items/d3f6c390ba6238a2f66d

In [50]:
from langchain_aws import ChatBedrock
from langchain_core.prompts import PromptTemplate

llm = ChatBedrock(
    model_id=model_id,
    region_name="us-east-1"
)

template = "①メンバーに関する内容乃木坂46×So-netコラボキャンペーンサイトでは、CM撮影時のオフショットやメンバーから②『6th Anniversary 乃木坂46時間TV』を忘れない画像"
prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

chain = prompt | llm

question = "次の文章を内容に基づいてサマリーを出してください。"
answer = chain.invoke({"question":question})
print(answer.content)

すみません。あなたの文章は完全な文ではないようで、いくつかの部分が欠けているように見えます。しかし、提供された情報から以下のように理解しました：

1. 乃木坂46とSo-netのコラボキャンペーンサイトについての言及があります。

2. このサイトでは、CMの撮影時のオフショット画像が公開されているようです。

3. 『6th Anniversary 乃木坂46時間TV』に関連する何らかの画像も含まれているようです。

4. これらの画像はメンバーから提供されたものかもしれません。

完全な文脈や詳細が不足しているため、正確な情報を提供するのは難しいですが、乃木坂46のファンにとって興味深いコンテンツが含まれているキャンペーンサイトについての言及だと推測されます。より詳細な情報や具体的な質問があれば、お知らせください。
